In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
!kaggle kernels output robikscube/fast-data-loading-and-low-mem-with-parquet-files/data?select=train_low_mem.parquet -p .

Kernel log downloaded to ./fast-data-loading-and-low-mem-with-parquet-files.log 


In [ ]:
#SKIP
!echo '{"username":"hackerinheels","key":"82bbcfeae33614a0dabfa494a6ec3573"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d robikscube/ubiquant-parquet -f train_low_mem.parquet

/bin/bash: /root/.kaggle/kaggle.json: No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


100% 3.06G/3.07G [00:24<00:00, 142MB/s]
100% 3.07G/3.07G [00:24<00:00, 134MB/s]


In [1]:
import pyarrow
import pyarrow.parquet

##Load data
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:

# Load the data
#df = pd.read_parquet('train.parquet')
df = pd.read_parquet('train_low_mem.parquet')
# Lets look at the data

In [3]:
df.shape

(3141410, 304)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
sns.distplot(df_filtered['f_0'])
plt.subplot(1,2,2)
sns.distplot(df_filtered['f_1'])
plt.show()

sns.boxplot(df_filtered['target'])


In [ ]:
df_filtered = df 

# Create correlation matrix
corr_matrix = df_filtered.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
final = df_filtered.drop(to_drop, axis=1, inplace=True)

In [ ]:
final = df
for i in range (0, 300):
  colName = "f_" + str(i)
  #print(colName + "  " + str(df_filtered.shape))
  q_low = final[colName].quantile(0.001)
  q_hi  = final[colName].quantile(0.999)  
  final = final[(final[colName] < q_hi) & (final[colName] > q_low)]


In [ ]:
with open(path, 'w', encoding = 'utf-8-sig') as f:
  final.to_parquet(f)


In [4]:
final = df
y = final.target
x = final
x = x.drop(['row_id', 'target'], axis=1)
y.shape
x.shape

(3141410, 302)

In [ ]:
final.head()

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624
5,0_9,0,9,1.505904,0.608855,1.369305,-0.761515,0.865860,-0.359269,-1.835762,...,0.333684,-1.095620,-0.335999,0.819155,-1.060166,-0.343812,-1.087009,0.077862,0.142943,-0.055550


In [7]:
y.head()

0   -0.300875
1   -0.231040
2    0.568807
3   -1.064780
4   -0.531940
Name: target, dtype: float32

In [9]:
y.shape


(3141410,)

In [ ]:

plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
sns.distplot(df_filtered['f_0'])
plt.subplot(1,2,2)
sns.distplot(df_filtered['f_1'])
plt.show()

sns.boxplot(df_filtered['target'])

In [5]:
import pickle
filename = 'mar_model_2.sav'


In [6]:
from sklearn.model_selection import train_test_split
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(x, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [7]:
print(score_dataset(X_train, X_valid, y_train, y_valid))


0.6646375362010147


In [10]:
import pickle
filename = 'randomforest_model.sav'

In [12]:
#with open('randomforest_model.sav','wb') as f:
#    pickle.dump(model,f)
filename = 'catboost_model.sav'
with open('catboost_model.sav','wb') as f:
    pickle.dump(catmodel,f)

In [8]:
import catboost
from catboost import CatBoostRegressor, Pool, cv

train_pool = Pool(X_train, y_train)
validation_pool = Pool(X_valid, y_valid)

catmodel = CatBoostRegressor(iterations=5000,  random_seed=42, early_stopping_rounds=50)

catmodel.fit(train_pool, eval_set=validation_pool, verbose=100, plot=True)
catmodel.score(X_valid,y_valid)



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.065217
0:	learn: 0.9186757	test: 0.9214050	best: 0.9214050 (0)	total: 476ms	remaining: 39m 38s
100:	learn: 0.9104463	test: 0.9136808	best: 0.9136808 (100)	total: 34.6s	remaining: 27m 56s
200:	learn: 0.9079422	test: 0.9116779	best: 0.9116779 (200)	total: 1m 8s	remaining: 27m 15s
300:	learn: 0.9055735	test: 0.9099931	best: 0.9099931 (300)	total: 1m 44s	remaining: 27m 10s
400:	learn: 0.9033799	test: 0.9085168	best: 0.9085168 (400)	total: 2m 23s	remaining: 27m 28s
500:	learn: 0.9015091	test: 0.9073528	best: 0.9073528 (500)	total: 3m 3s	remaining: 27m 26s
600:	learn: 0.8997394	test: 0.9062725	best: 0.9062725 (600)	total: 3m 44s	remaining: 27m 20s
700:	learn: 0.8980883	test: 0.9052770	best: 0.9052770 (700)	total: 4m 26s	remaining: 27m 14s
800:	learn: 0.8965535	test: 0.9044496	best: 0.9044496 (800)	total: 5m 8s	remaining: 26m 55s
900:	learn: 0.8951039	test: 0.9037084	best: 0.9037084 (900)	total: 5m 50s	remaining: 26m 36s
1000:	learn: 0.8937383	test: 0.9030436	best: 0.90

0.06675346173969654

In [ ]:
from transformers import TrainingArguments,Trainer
bs = 128
epochs = 4
lr = 8e-5
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

trainer.train()
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

In [9]:
ax=catmodel.get_feature_importance(type='PredictionValuesChange',prettified=True)

fig = px.bar(ax, x='Feature Id',y=['Importances'],
              title="Cat Boost predictors importance - Prediction value change",
              color_discrete_sequence=px.colors.qualitative.Prism).update_layout( 
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)')

fig.show()

NameError: name 'px' is not defined

In [ ]:
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

regressor = TabNetRegressor(verbose=0,seed=42)


In [ ]:
import gc
gc.collect()

202

In [ ]:
df_filtered.corrwith(df_filtered['target']).plot(kind='barh',figsize=[200,100], legend=True,grid=True)


In [ ]:
tscv = TimeSeriesSplit(n_splits = 20)
rmse = []
CV_score_array    =[]
predictions_array =[]

for train_index, test_index in tscv.split(x):
    print(train_index, test_index)
    X_train, X_valid = x.iloc[train_index], x.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

    X_train      = X_train.to_numpy()
    y_train = y_train.to_numpy().reshape(-1, 1)
    
    X_valid      = X_valid.to_numpy()
    y_valid = y_valid.to_numpy().reshape(-1, 1)
    
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=100,
              eval_metric=['rmse'])
    predictions = regressor.predict(X_valid)
    pickle.dump(regressor, open(filename, 'wb'))
    CV_score_array.append(regressor.best_cost)
    print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )
    predictions_array.append(np.expm1(predictions))


!cp filename "/content/drive/My Drive/"

[    0     1     2 ... 81968 81969 81970] [ 81971  81972  81973 ... 163938 163939 163940]
Stop training because you reached max_epochs = 100 with best_epoch = 1 and best_val_0_rmse = 0.7871699929237366
Best weights from best epoch are automatically used!
The CV score is 0.78717
[     0      1      2 ... 163938 163939 163940] [163941 163942 163943 ... 245908 245909 245910]


In [ ]:
!cp filename "/content/drive/My Drive/"


In [ ]:
# load the model from disk
regressor = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)


Mounted at /content/drive
